# NOAA Sea Surface Temperature (SST) Data Processing


In [33]:
import xarray as xr
print(xr.__version__) # should output 2023.12.0

import netCDF4
print(netCDF4.__version__) # should output 1.7.2

from xarray.backends import list_engines
print(list_engines()) # check that 'netcdf4' or 'nc4' is part of the engine

# import pandas and altair for further processing & visualization
import pandas as pd
import altair as alt
alt.data_transformers.enable("vegafusion") ## for big data sets

import os # to read many files from the operating system


2023.12.0
1.7.2
{'netcdf4': <NetCDF4BackendEntrypoint>
  Open netCDF (.nc, .nc4 and .cdf) and most HDF5 files using netCDF4 in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.NetCDF4BackendEntrypoint.html, 'h5netcdf': <H5netcdfBackendEntrypoint>
  Open netCDF (.nc, .nc4 and .cdf) and most HDF5 files using h5netcdf in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.H5netcdfBackendEntrypoint.html, 'scipy': <ScipyBackendEntrypoint>
  Open netCDF files (.nc, .nc4, .cdf and .gz) using scipy in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.ScipyBackendEntrypoint.html, 'store': <StoreBackendEntrypoint>
  Open AbstractDataStore instances in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.StoreBackendEntrypoint.html}


## 1. Exploring the Data Structure

First we open one example file to get to know the netCDF4 format a bit better. 

In [32]:
# load one file and print the data
ds = xr.open_dataset("./data/oisst-avhrr-v02r01.20250101_example.nc", engine='netcdf4')
print(ds)

<xarray.Dataset>
Dimensions:  (time: 1, zlev: 1, lat: 720, lon: 1440)
Coordinates:
  * time     (time) datetime64[ns] 2025-01-01T12:00:16.364011520
  * zlev     (zlev) float32 0.0
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
Data variables:
    sst      (time, zlev, lat, lon) float32 ...
    anom     (time, zlev, lat, lon) float32 ...
    err      (time, zlev, lat, lon) float32 ...
    ice      (time, zlev, lat, lon) float32 ...
Attributes: (12/37)
    Conventions:                CF-1.6, ACDD-1.3
    title:                      NOAA/NCEI 1/4 Degree Daily Optimum Interpolat...
    references:                 Reynolds, et al.(2007) Daily High-Resolution-...
    source:                     ICOADS, NCEP_GTS, GSFC_ICE, NCEP_ICE, Pathfin...
    id:                         oisst-avhrr-v02r01.20250101.nc
    naming_authority:           gov.noaa.ncei
    ...                         ...

SST (sea surface temperature) is 4-dimensional:
- time: 1 time point (for now, just the 20250101 file)
- zlev: 1 depth level (surface = 0m)
- lat and lon: your spatial grid

In [69]:
# get the sst data section and print it
sst = ds['sst']
print(sst)

<xarray.DataArray 'sst' (time: 1, zlev: 1, lat: 720, lon: 1440)>
[1036800 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2023-01-01T11:59:54.563395584
  * zlev     (zlev) float32 0.0
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
Attributes:
    long_name:  Daily sea surface temperature
    units:      Celsius
    valid_min:  -300
    valid_max:  4500


### Extract the SST for a specific region (bounding box)

To focus on a region like the North Atlantic, you can slice the dataset like this:

```python
northAtlantic = ds.sst.sel(lat=slice(30, 60), lon=slice(-80, 0))
```

Attention: Many climate and ocean datasets, including NOAA’s OISST, store longitude in the range [0, 360], instead of the more intuitive -180° to 180° format (where negative = west).
For example:

- 280°E is equivalent to -80°W.
- 359.9°E is just shy of 0°.

Therefore the northAtlantic would be: `lat=slice(30, 60), lon=slice(280, 360)`. You can check the format with `print(ds.lon.values)`

To handle this in Xarray, you can shift longitudes to [0, 360] like this: `ds = ds.assign_coords(lon=(ds.lon % 360))` (ChatGPT untested).

Ideally use QGIS or some tool to define your bounding box.

In [19]:
northAtlantic = ds.sst.sel(lat=slice(30, 60), lon=slice(280, 360))
print(northAtlantic)

<xarray.DataArray 'sst' (time: 1, zlev: 1, lat: 120, lon: 320)>
[38400 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2025-01-01T12:00:16.364011520
  * zlev     (zlev) float32 0.0
  * lat      (lat) float32 30.12 30.38 30.62 30.88 ... 59.12 59.38 59.62 59.88
  * lon      (lon) float32 280.1 280.4 280.6 280.9 ... 359.1 359.4 359.6 359.9
Attributes:
    long_name:  Daily sea surface temperature
    units:      Celsius
    valid_min:  -300
    valid_max:  4500


To see the actual temperature values you need to call the following. That output is the sea surface temperature (SST) in Celsius.

- Warm water (25°C) near the southern end of your region.
- Colder water (around 8-9°C) closer to 60°N.
- Some nan values (no data or land areas).

In [20]:
print(northAtlantic.values)

[[[[25.82       25.849998   25.46       ...         nan         nan
            nan]
   [25.699999   25.869999   25.599998   ...         nan         nan
            nan]
   [25.25       25.789999   25.75       ...         nan         nan
            nan]
   ...
   [-0.5        -0.17999999  0.06       ...  8.74        8.67
     8.54      ]
   [-0.74       -0.42999998 -0.14       ...  8.84        8.82
     8.72      ]
   [-0.89       -0.56       -0.21       ...  8.9         8.94
     8.9       ]]]]


### Transform the Values into a Pandas Dataframe

In [41]:
# Convert to DataFrame
df = northAtlantic.to_dataframe().reset_index()
df.head()

# Drop missing (NaN) values if needed (not tested, ChatGPT)
# df = df.dropna(subset=['sst'])

# close the dataset to free memory
ds.close()

## 2. Scaling Up

So far so good. However, the matter is more complex. The files from the [NOAA directory](https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/) are for each day of the month for each year (1981 to 2025). We need to calculate a yearly average for each year. For each month of the year we can take the first day and then calculate the average of the year.

In [66]:
# set the year you want to calculate an combined file, change this for the next year!
set_year = '2023'

# create the folder path automatically
set_dir = './data/' + set_year + '/'

# List files and sort by date embedded in filename
files_2023 = sorted(
    [os.path.join(set_dir, file) for file in os.listdir(set_dir) if file.endswith('.nc')],
    key=lambda x: x.split('.')[-2]  # This grabs the YYYYMMDD part right before '.nc'
)

# Optional: print to verify order
for f in files_2023:
    print(f)


./data/2023/oisst-avhrr-v02r01.20230101.nc
./data/2023/oisst-avhrr-v02r01.20230201.nc
./data/2023/oisst-avhrr-v02r01.20230301.nc
./data/2023/oisst-avhrr-v02r01.20230401.nc
./data/2023/oisst-avhrr-v02r01.20230501.nc
./data/2023/oisst-avhrr-v02r01.20230601.nc
./data/2023/oisst-avhrr-v02r01.20230701.nc
./data/2023/oisst-avhrr-v02r01.20230801.nc
./data/2023/oisst-avhrr-v02r01.20230901.nc
./data/2023/oisst-avhrr-v02r01.20231001.nc
./data/2023/oisst-avhrr-v02r01.20231101.nc
./data/2023/oisst-avhrr-v02r01.20231201.nc


Now that we have all files from one year loaded, we can iterate over each file using a for loop and save each files data into a Pandas Dataframe.

In [67]:
# Initialize an empty list to store the data of the year
# for each file we open, we will save a data frame into this
# it will be a list of data frames
data_list = []

# Iterate over each file you loaded before
for file in files_2023:
    # Open the current file/dataset
    current_ds = xr.open_dataset(file)

    # Security check if 'sst' is in the file
    if 'sst' not in current_ds.variables:
        print(f"Warning: 'sst' not found in {file}")
        continue  # Skip file if variable is missing

    # select the area (north american ocean)
    area_select = current_ds.sst.sel(lat=slice(30, 60), lon=slice(280, 360))

    # get the values info a data frame
    new_df = area_select.to_dataframe().reset_index()

    # Add year column to keep track of the current year
    new_df['year'] = set_year

    # append the dataframe with the data of the current file to the combined list
    data_list.append(new_df)
    
    # Close the dataset to free up memory
    current_ds.close()


# Now we have one big list of dataframes, one frame for each file in the year directory
# now we combine all of them into one dataframe for further processing
final_df = pd.concat(data_list, ignore_index=True)

# we can see the lenght and first entries of this large dataframe
print(len(final_df))
# Show the first few rows of the dataframe
print(final_df.head(3))

# Now we need to create a mean for each position (long/lat), as every position is now multiple times in the frame (one time for each file)
# We can do this by grouping the entries of the data frame by year, lat and log and then calculate the mean using Pandas
mean_sst_df = final_df.groupby(['year', 'lat', 'lon'])['sst'].mean().reset_index()

# yaay!
print(len(mean_sst_df))
print(mean_sst_df)

460800
                           time  zlev     lat      lon        sst  year
0 2023-01-01 11:59:54.563395584   0.0  30.125  280.125  25.709999  2023
1 2023-01-01 11:59:54.563395584   0.0  30.125  280.375  25.830000  2023
2 2023-01-01 11:59:54.563395584   0.0  30.125  280.625  25.730000  2023
38400
       year     lat      lon        sst
0      2023  30.125  280.125  27.776667
1      2023  30.125  280.375  27.776667
2      2023  30.125  280.625  27.362497
3      2023  30.125  280.875  26.847498
4      2023  30.125  281.125  26.438334
...     ...     ...      ...        ...
38395  2023  59.875  358.875  10.227500
38396  2023  59.875  359.125  10.373333
38397  2023  59.875  359.375  10.501666
38398  2023  59.875  359.625  10.549167
38399  2023  59.875  359.875  10.521667

[38400 rows x 4 columns]


## Export

In [68]:
output_dir = './export/'

final_df.to_csv(f'{output_dir}north_atlantic_{set_year}.csv', index=False)